In [1]:
import os
import cv2
import random
import shutil
import xml.dom.minidom
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

### move jpgs with xmls/txts out

In [3]:
def scan_files(directory, prefix=None, postfix=None):
    files_list = []
    for root, sub_dirs, files in os.walk(directory):
        for special_file in files:
            if postfix:
                if special_file.endswith(postfix):
                    files_list.append(os.path.join(root, special_file))
            elif prefix:
                if special_file.startswith(prefix):
                    files_list.append(os.path.join(root, special_file))
            else:
                files_list.append(os.path.join(root, special_file))
    return files_list

def copy_jpg_and_labels(from_path, to_path):
    image_names = scan_files(from_path, postfix=".jpg")
    for image_name in image_names:
        xml_name = os.path.splitext(image_name)[0] + ".xml"
        txt_name = os.path.splitext(image_name)[0] + ".txt"
        if os.path.isfile(xml_name):
            shutil.move(image_name, to_path)
            shutil.move(xml_name, to_path)
        elif os.path.isfile(txt_name):
            shutil.move(image_name, to_path)
            shutil.move(txt_name, to_path)
            

orig_path = "../../yolo_tri_20181128/TRI-origin"
tmp_path = "../../yolo_tri_20181128/TRI-tmp"

copy_jpg_and_labels(orig_path, tmp_path)

### find proper training input image size (608x608)

In [9]:
# collect all sizes of images
image_path = "../../yolo_tri_20181128/TRI-tmp"
image_names = [os.path.join(image_path, f) for f in os.listdir(image_path) if f.endswith(".jpg")]
print("# of jpgs", len(image_names))
image_sizes = []
for image_name in image_names:
    with Image.open(image_name) as image:
        image_sizes.append(image.size)

# of jpgs 21499


In [10]:
# kmeans clustering
X = np.array(image_sizes)
kmeans = KMeans(n_clusters=9, random_state=2018).fit(X)
print(X.shape, kmeans)

(21499, 2) KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=9, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=2018, tol=0.0001, verbose=0)


In [7]:
centers = []
for center in kmeans.cluster_centers_:
    print(center)
    centers.append(center)

[58.27678161 59.98114943]
[97.95093229 83.609421  ]
[ 90.74608905 105.70397112]
[64.63528315 69.58750874]
[75.79792463 83.64281813]
[78.86837227 66.36707453]
[50.55930569 48.26229508]
[68.27896411 57.10949568]
[360.5 274. ]


In [8]:
total = len(image_sizes)
large = len([image_size for image_size in image_sizes if image_size[0] > 608 or image_size[1] > 608])
print(total, large)

21499 0


### image padding and resizing

In [15]:
def read_labels_from_xml(xml_name):
    DOMTree = xml.dom.minidom.parse(xml_name)
    collection = DOMTree.documentElement
    filename = collection.getElementsByTagName("filename")
    objects = collection.getElementsByTagName("object")

    w = collection.getElementsByTagName("width")[0]
    w_val = int(w.firstChild.nodeValue)
    h = collection.getElementsByTagName("height")[0]
    h_val = int(h.firstChild.nodeValue)
    w.firstChild.replaceWholeText(str(h_val))
    h.firstChild.replaceWholeText(str(w_val))

    labels = {'w':w_val, 'h':h_val, 'boxes':[]}
    
    for object in objects:
        xmin = object.getElementsByTagName("xmin")[0]
        xmin_val = int(xmin.firstChild.nodeValue)
        xmax = object.getElementsByTagName("xmax")[0]
        xmax_val = int(xmax.firstChild.nodeValue)
        ymin = object.getElementsByTagName("ymin")[0]
        ymin_val = int(ymin.firstChild.nodeValue)
        ymax = object.getElementsByTagName("ymax")[0]
        ymax_val = int(ymax.firstChild.nodeValue)
        labels["boxes"].append([xmin_val, ymin_val, xmax_val, ymax_val])
        
    return labels


def read_labels_from_txt(txt_name):
    jpg_name = os.path.splitext(txt_name)[0] + ".jpg"
    with Image.open(jpg_name) as img:
        w, h = img.size
        
    labels = {'w':w, 'h':h, 'boxes':[]}
    with open(txt_name, 'r') as f:
        for line in f.readlines():
            tokens = line.strip().split()
            xc, yc = w*float(tokens[1]), h*float(tokens[2])
            w_, h_ = w*float(tokens[3]), h*float(tokens[4])
            xmin, ymin = int(xc - w_/2), int(yc - h_/2)
            xmax, ymax = int(xc + w_/2), int(yc + h_/2)
            labels["boxes"].append([xmin, ymin, xmax, ymax])
            
    return labels


def resize_img_with_padding(img_name, size, save_path):
    xml_name = os.path.splitext(img_name)[0] + ".xml"
    txt_name = os.path.splitext(img_name)[0] + ".txt"
    if os.path.isfile(xml_name):
        labels = read_labels_from_xml(xml_name)
    elif os.path.isfile(txt_name):
        labels = read_labels_from_txt(txt_name)
    else:
        return
    
    img_name_new = os.path.join(save_path, os.path.basename(img_name))
    txt_name_new = os.path.splitext(img_name_new)[0] + ".txt"
    
    img = Image.open(img_name)
    
    # pad and crop image
    x = -((size - img.size[0]) // 2)
    y = -((size - img.size[1]) // 2)
    img_croped = img.crop(
        (
            x,
            y,
            size + x,
            size + y
        )
    )
    img_croped.save(img_name_new)

    # change label coordinates
    labels_yolo = []
    for box in labels["boxes"]:
        xmin = box[0] - x
        ymin = box[1] - y
        xmax = box[2] - x
        ymax = box[3] - y
        xcenter = (xmin+xmax)/2.0/size
        ycenter = (ymin+ymax)/2.0/size
        w = (xmax-xmin)/size
        h = (ymax-ymin)/size
        labels_yolo.append(['0', str(xcenter), str(ycenter), str(w), str(h)])
        
#         # cut cells
#         cell_name = os.path.splitext(img_name_new)[0] + '_' + str(xmin) + '_' + str(ymin) + ".jpg"
#         img_croped.crop((xmin, ymin, xmax, ymax)).save(cell_name)
        
    # write lables to txt
    with open(txt_name_new, 'w') as f:
        for label in labels_yolo:
            f.write(' '.join(label) + '\n')
            
    print("processed ", img_name)

In [16]:
tmp_path = "../../yolo_tri_20181128/TRI-tmp"
yolo_path = "../../yolo_tri_20181128/TRI-yolo"
size = 608

image_names = [os.path.join(tmp_path, f) for f in os.listdir(tmp_path) if f.endswith(".jpg")]

for image_name in image_names:
    resize_img_with_padding(image_name, size, yolo_path)

processed  ../../yolo_tri_20181128/TRI-tmp/2018-06-01_12_40_20_x11636_y33305_w76_h72.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/2018-01-15-15_39_28_x40641_y11131_w68_h98.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18052625_x19048_y26314_w68_h64.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/2018-06-01_12_40_20_x11800_y39758_w82_h58.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/2018-01-15-15_32_42_x10507_y24433_w92_h94.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18053765_x16406_y44064_w62_h74.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/2018-06-01_12_40_20_x12108_y31741_w68_h52.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18056790_x11758_y9191_w72_h76.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/2018-05-08-16_19_25_x16475_y29475_w78_h60.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/2018-05-08-16_43_07_x38601_y28593_w56_h50.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18014088_x25768_y25288_w54_h56.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18056789_x283

processed  ../../yolo_tri_20181128/TRI-tmp/TC18056751_x8298_y40665_w66_h58.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18056268_x49443_y16938_w66_h64.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/2018-06-01_12_40_20_x14069_y45874_w62_h72.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18050519_x27943_y14894_w70_h60.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18056268_x39513_y40273_w70_h72.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18056383_x25114_y35160_w66_h62.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/2018-01-15-15_49_03_x13350_y20353_w78_h72.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/2018-05-08-16_43_07_x35286_y39488_w50_h52.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18014088_x43529_y13385_w70_h60.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/2018-01-15-15_51_25_x26795_y32936_w90_h100.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/2018-01-15-15_13_04_x13275_y17124_w88_h68.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18055080_x36078_y25510_w78_h72

processed  ../../yolo_tri_20181128/TRI-tmp/2018-01-15-15_49_03_x40287_y38110_w74_h62.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18050519_x6937_y18134_w52_h48.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18052588_x38904_y30590_w66_h50.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/2018-01-15-15_23_26_x50419_y28602_w88_h74.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18056383_x31934_y19806_w74_h64.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18050519_x11533_y24402_w48_h44.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/2018-05-08-16_45_06_x6196_y33356_w50_h50.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/2018-01-15-15_32_42_x9930_y22916_w90_h108.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18053451_x32646_y51478_w58_h58.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/2018-06-01_12_40_20_x10575_y17554_w74_h62.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18055080_x18247_y33802_w76_h64.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/NJ18088753_x49626_y41044_w76_h62.j

processed  ../../yolo_tri_20181128/TRI-tmp/TC18052625_x39309_y45381_w60_h60.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18053451_x11341_y16414_w62_h60.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18054797_x17828_y15814_w48_h46.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18018836_x17749_y23215_w58_h66.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/2018-05-08-16_43_07_x44870_y21689_w44_h52.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18056383_x41371_y7636_w62_h62.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18053589_x25127_y10623_w70_h70.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/2018-01-15-15_49_03_x48562_y32862_w102_h94.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18056497_x26973_y27079_w72_h56.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/2018-05-08-16_13_03_x15813_y17452_w110_h72.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/2018-01-15-17_41_06_x29027_y27714_w70_h68.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18053451_x40103_y29373_w60_h62.jpg
pro

processed  ../../yolo_tri_20181128/TRI-tmp/2018-01-15-15_49_03_x22812_y19270_w98_h94.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/NJ18088753_x47394_y18577_w68_h68.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/2018-01-15-17_41_06_x38420_y14954_w80_h68.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/2018-05-08-16_43_07_x24652_y24267_w66_h62.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18053589_x33245_y32912_w58_h70.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18055080_x19492_y35256_w68_h64.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18056383_x30573_y46743_w62_h60.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/2018-05-08-16_43_07_x42461_y12137_w50_h58.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/NJ18088753_x44800_y40043_w74_h58.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18052588_x38707_y45561_w64_h60.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/2018-06-01_12_40_20_x18227_y33601_w54_h64.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18018836_x55311_y29997_w66_h60

...

processed  ../../yolo_tri_20181128/TRI-tmp/2018-01-15-15_13_04_x21857_y7327_w74_h68.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18055080_x28538_y12332_w62_h68.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18050519_x15000_y25850_w70_h66.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18053451_x16220_y39331_w64_h58.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18018836_x37294_y15567_w68_h54.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/2018-05-08-16_19_25_x52052_y42558_w56_h58.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/NJ18088753_x31046_y45335_w62_h66.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/2018-05-08-16_26_21_x53146_y31057_w56_h54.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18056497_x19446_y46516_w70_h78.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18055080_x33411_y28990_w66_h62.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18014088_x9764_y32597_w70_h72.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18053634_x45984_y17139_w48_h40.jpg
processed  ../.

processed  ../../yolo_tri_20181128/TRI-tmp/TC18054797_x10919_y22063_w42_h40.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18052625_x11626_y35443_w68_h62.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18054797_x41134_y54536_w44_h38.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18056268_x46358_y11203_w76_h58.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18056383_x31383_y18309_w70_h80.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/2018-01-15-17_41_06_x18886_y36501_w78_h62.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/2018-06-01_12_40_20_x14819_y37098_w54_h60.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/2018-01-15-15_51_25_x39037_y48888_w82_h78.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18053285_x45066_y44883_w70_h54.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/NJ18088753_x42110_y16735_w60_h62.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18055080_x42771_y25277_w50_h66.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18053765_x45133_y22647_w58_h62.jpg
processed  ..

processed  ../../yolo_tri_20181128/TRI-tmp/2018-01-15-15_13_04_x46005_y29111_w80_h82.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18053285_x49919_y50996_w76_h66.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18014088_x37734_y44163_w64_h62.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18014088_x34642_y32598_w46_h42.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/2018-05-08-16_43_07_x21865_y30507_w58_h60.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/2018-01-15-15_13_04_x35692_y26440_w98_h62.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18014088_x22893_y34462_w66_h70.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/2018-06-01_12_40_20_x16910_y48211_w58_h58.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/2018-01-15-17_41_06_x29802_y33330_w72_h68.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18053765_x42333_y24273_w62_h54.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18056790_x25514_y8246_w70_h68.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18056268_x19840_y32690_w70_h60.

processed  ../../yolo_tri_20181128/TRI-tmp/TC18018836_x49721_y14269_w80_h58.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18053451_x10148_y47412_w62_h68.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18056789_x40939_y16081_w60_h48.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18014088_x35270_y29380_w66_h64.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18053765_x22299_y12044_w58_h72.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/2018-05-08-17_01_57_x23854_y10486_w66_h60.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/2018-01-15-15_37_12_x15537_y13990_w88_h94.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18052625_x42772_y28336_w46_h44.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18056268_x10948_y14074_w78_h80.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18014088_x16794_y7082_w68_h60.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18014088_x28606_y36901_w58_h58.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18014088_x11594_y32820_w54_h60.jpg
processed  ../../yolo_t

processed  ../../yolo_tri_20181128/TRI-tmp/TC18014088_x15602_y32076_w68_h58.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/2018-06-01_12_40_20_x13328_y40463_w84_h68.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/2018-06-01_12_26_15_x46132_y8560_w50_h62.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18056790_x22876_y18760_w64_h62.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18018836_x33618_y35579_w56_h58.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18055080_x32229_y34659_w76_h58.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18056790_x23498_y28715_w78_h102.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18053451_x26508_y25351_w58_h62.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/TC18014088_x44990_y20843_w64_h64.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/2018-01-15-17_41_06_x16973_y20860_w80_h68.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/NJ18088753_x33025_y17948_w66_h70.jpg
processed  ../../yolo_tri_20181128/TRI-tmp/2018-06-01_12_40_20_x17882_y13499_w72_h72.jpg
proc

### split data into train/valid

In [17]:
import random
import shutil


def split_train_valid(data_path):
    img_names = [os.path.join(data_path, f) for f in os.listdir(data_path) if f.endswith(".jpg")]

    random.shuffle(img_names)
    random.shuffle(img_names)

    split = 0.8

    train_names = img_names[:int(len(img_names)*split)]
    valid_names = img_names[int(len(img_names)*split):]

    train_path = os.path.join(data_path, "train")
    os.makedirs(train_path, exist_ok=True)
    for img_name in train_names:
        txt_name = os.path.splitext(img_name)[0] + ".txt"
        shutil.move(img_name, train_path)
        shutil.move(txt_name, train_path)

    valid_path = os.path.join(data_path, "valid")
    os.makedirs(valid_path, exist_ok=True)
    for img_name in valid_names:
        txt_name = os.path.splitext(img_name)[0] + ".txt"
        shutil.move(img_name, valid_path)
        shutil.move(txt_name, valid_path)
        

yolo_path = "../../yolo_tri_20181128/TRI-yolo"
split_train_valid(yolo_path)

### kmeans clustering

In [20]:
def scan_files(directory, prefix=None, postfix=None):
    files_list = []
    for root, sub_dirs, files in os.walk(directory):
        for special_file in files:
            if postfix:
                if special_file.endswith(postfix):
                    files_list.append(os.path.join(root, special_file))
            elif prefix:
                if special_file.startswith(prefix):
                    files_list.append(os.path.join(root, special_file))
            else:
                files_list.append(os.path.join(root, special_file))
    return files_list

def collect_sizes_from_yolotxt(txt_name, size):
    sizes = []
    with open(txt_name, 'r') as f:
        for line in f.readlines():
            tokens = line.strip().split()
#             print(tokens)
            w = int(float(tokens[3])*size)
            h = int(float(tokens[4])*size)
            sizes.append([w, h])
    return sizes

def collect_sizes_from_yolotxt_all(txt_names, size):
    sizes = []
    for txt_name in txt_names:
        sizes += collect_sizes_from_yolotxt(txt_name, size)
    return sizes

In [21]:
yolo_path = "../../yolo_tri_20181128/TRI-yolo"

sizes = collect_sizes_from_yolotxt_all(scan_files(yolo_path, postfix=".txt"), 608)

# kmeans clustering
X = np.array(sizes)
kmeans = KMeans(n_clusters=9, random_state=2018).fit(X)
print(kmeans)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=9, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=2018, tol=0.0001, verbose=0)


In [22]:
centers = []
for center in kmeans.cluster_centers_:
    print(center)
    centers.append(center)

[22.86953015 26.52934059]
[28.34621756 33.36443954]
[37.32348993 45.84832215]
[220.5  148.25]
[29.80449018 25.72747116]
[24.75580068 21.99410422]
[49.99404762 39.9265873 ]
[19.67013798 20.48633168]
[38.24017094 32.6957265 ]


In [23]:
tosort = {int(center[0]*center[1]):center for center in centers}
print(tosort)

{544: array([24.75580068, 21.99410422]), 945: array([28.34621756, 33.36443954]), 402: array([19.67013798, 20.48633168]), 606: array([22.86953015, 26.52934059]), 32689: array([220.5 , 148.25]), 1996: array([49.99404762, 39.9265873 ]), 1250: array([38.24017094, 32.6957265 ]), 766: array([29.80449018, 25.72747116]), 1711: array([37.32348993, 45.84832215])}


In [24]:
hassorted = sorted(tosort.items())
print(hassorted)
print(",  ".join(["{},{}".format(int(value[1][0]),int(value[1][1])) for value in hassorted]))

[(402, array([19.67013798, 20.48633168])), (544, array([24.75580068, 21.99410422])), (606, array([22.86953015, 26.52934059])), (766, array([29.80449018, 25.72747116])), (945, array([28.34621756, 33.36443954])), (1250, array([38.24017094, 32.6957265 ])), (1711, array([37.32348993, 45.84832215])), (1996, array([49.99404762, 39.9265873 ])), (32689, array([220.5 , 148.25]))]
19,20,  24,21,  22,26,  29,25,  28,33,  38,32,  37,45,  49,39,  220,148


### cut cells from yolo images/txts

In [ ]:
import os
from PIL import Image

image_dir = "./CC_yolo/train"
save_path = "./CC_cell"

def cut_cell(img_name, txt_name, save_path):
    img = Image.open(img_name)
    w,h = img.size
    labels = []
    with open(txt_name, 'r') as f:
        for line in f.readlines():
            tokens = line.strip().split(',')
            cx, cy = float(tokens[1]), float(tokens[2])
            w_, h_ = float(tokens[3]), float(tokens[4])
            xmin, ymin = int((cx-w_/2)*w), int((cy-h_/2)*h)
            xmax, ymax = int((cx+w_/2)*w), int((cy+h_/2)*h)
            img.crop((xmin, ymin, xmax, ymax)).save(os.path.join(save_path, "{}_{}_{}_{}.jpg".format(xmin, ymin, xmax, ymax)))
    img.close()
    
img_names = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(".jpg")]
for img_name in img_names:
    txt_name = os.path.splitext(img_name)[0] + ".txt"
    cut_cell(img_name, txt_name, save_path)